In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from PIL import Image
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.optim import Adam
import pandas as pd
import matplotlib.patches as patches

from GlaucomaDataset import GlaucomaDatasetBoundingBoxes
from unet import UNet

In [2]:
origa_path = os.path.join("..", '..', "data", "ORIGA")
images_path = os.path.join(origa_path, "Images_Square")
masks_path = os.path.join(origa_path, "Masks_Square")

img_filenames = sorted(os.listdir(images_path))
mask_filenames = sorted(os.listdir(masks_path))

In [3]:
def update_image_path(path):
    split_path = path.split("/")
    return split_path[-1]

In [4]:
bb_df = pd.read_csv("../../data/ORIGA/bounding_boxes.csv")
bb_df['image_path'] = bb_df['image_path'].apply(update_image_path)
bb_df[['x1', 'y1', 'x2', 'y2']] //= 2

In [5]:
# Split into train, validation, and test sets (70, 15, 15)
train_imgs, temp_imgs, train_masks, temp_masks = train_test_split(
    img_filenames, mask_filenames, test_size=0.3, random_state=42)

val_imgs, test_imgs, val_masks, test_masks = train_test_split(
    temp_imgs, temp_masks, test_size=0.5, random_state=42)

In [6]:
# Load data
batch_size = 8
n_workers = 4

train_set = GlaucomaDatasetBoundingBoxes(images_path, masks_path, train_imgs, train_masks, bb_df)
val_set = GlaucomaDatasetBoundingBoxes(images_path, masks_path, val_imgs, val_masks, bb_df)
test_set = GlaucomaDatasetBoundingBoxes(images_path, masks_path, test_imgs, test_masks, bb_df)

train_loader = DataLoader(train_set, batch_size=batch_size, num_workers=n_workers, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, num_workers=n_workers, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, num_workers=n_workers, shuffle=True)

In [55]:
def display_image_with_bbox(img_array, bbox, color='red', linewidth=2):
    """
    Display an image with a bounding box overlay.
    
    Parameters:
    -----------
    img_array : numpy.ndarray
        Image as a numpy array.
    bbox : list or tuple
        Bounding box coordinates in the format [x1, y1, x2, y2]
        where (x1, y1) is the top-left corner and (x2, y2) is the bottom-right corner.
    color : str, default='red'
        Color of the bounding box.
    linewidth : int, default=2
        Width of the bounding box border.
    """
    # Create figure and axes
    fig, ax = plt.subplots(1)
    
    # Display the image
    ax.imshow(img_array)
    
    # Extract coordinates
    x1, y1, x2, y2 = bbox
    
    # Calculate width and height for the rectangle
    width = x2 - x1
    height = y2 - y1
    
    # Create a Rectangle patch
    rect = patches.Rectangle((x1, y1), width, height, linewidth=linewidth, 
                             edgecolor=color, facecolor='none')
    
    # Add the rectangle to the plot
    ax.add_patch(rect)
    
    # Remove axis ticks and labels
    ax.set_xticks([])
    ax.set_yticks([])
    
    # Show the plot
    plt.tight_layout()
    plt.show()

## Define Metrics

In [9]:
def dice_coefficient(targets, preds, smooth=1e-6):
    # preds = (preds > 0.5).float 
    intersection = torch.sum(preds * targets, dim=(2,3))
    # want close to 1 (identical)
    dice = (2. * intersection + smooth) / (torch.sum(preds, dim=(2,3)) + torch.sum(targets, dim=(2,3)) + smooth)
    return dice.mean()

## Train and Test Loops

In [10]:
def trainloop(dataloader, model, loss_func, optimizer):
    num_batches = len(dataloader)
    train_loss, dice = 0., 0. 
    
    for image, image_name, mask, mask_name in dataloader:
        image, mask = image.to(device), mask.to(device)
        
        optimizer.zero_grad()
        pred = model(image)
        loss = loss_func(pred, mask)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        dice += dice_coefficient(pred, mask).item()
    train_loss /= num_batches
    dice /= num_batches
        
    return train_loss, dice

def testloop(dataloader, model, loss_func):
    num_batches = len(dataloader)
    test_loss, dice = 0. , 0.
    
    with torch.no_grad():
        for image, image_name, mask, mask_name in dataloader:
            image, mask = image.to(device), mask.to(device)
            pred = model(image)
            test_loss += loss_func(pred, mask).item()
            dice += dice_coefficient(pred, mask).item()
    test_loss /= num_batches
    dice /= num_batches
    
    return test_loss, dice

## Initalize and Train Model

In [14]:
# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UNet(in_channels=4).to(device)

lr = 1e-4
loss_func = torch.nn.BCELoss(reduction='mean')
optimizer = Adam(model.parameters(), lr=lr)

In [16]:
# Training and validation
epochs = 60

for epoch in range(epochs):
    
    model.train()
    print(f"---Epoch {epoch+1}/{epochs}:---")
    train_loss, dice = trainloop(train_loader, model, loss_func, optimizer)
    print(f"Train Loss: {train_loss:.4f}   Train Dice: {dice:.4f}")
    
    model.eval()
    test_loss, dice = testloop(val_loader, model, loss_func)
    print(f"Test Error: \n Avg Loss: {test_loss:>8f}  Test Dice: {dice:.4f} \n")

---Epoch 1/60:---
Train Loss: 0.2911   Train Dice: 0.0087
Test Error: 
 Avg Loss: 0.059984  Test Dice: 0.0113 

---Epoch 2/60:---
Train Loss: 0.0345   Train Dice: 0.1473
Test Error: 
 Avg Loss: 0.024065  Test Dice: 0.2704 

---Epoch 3/60:---
Train Loss: 0.0239   Train Dice: 0.3119
Test Error: 
 Avg Loss: 0.023716  Test Dice: 0.3722 

---Epoch 4/60:---
Train Loss: 0.0217   Train Dice: 0.3790
Test Error: 
 Avg Loss: 0.021007  Test Dice: 0.4197 

---Epoch 5/60:---
Train Loss: 0.0215   Train Dice: 0.3913
Test Error: 
 Avg Loss: 0.018371  Test Dice: 0.4708 

---Epoch 6/60:---
Train Loss: 0.0166   Train Dice: 0.5155
Test Error: 
 Avg Loss: 0.013351  Test Dice: 0.5776 

---Epoch 7/60:---
Train Loss: 0.0131   Train Dice: 0.5996
Test Error: 
 Avg Loss: 0.011900  Test Dice: 0.6781 

---Epoch 8/60:---
Train Loss: 0.0114   Train Dice: 0.6413
Test Error: 
 Avg Loss: 0.009831  Test Dice: 0.6751 

---Epoch 9/60:---
Train Loss: 0.0104   Train Dice: 0.6707
Test Error: 
 Avg Loss: 0.012103  Test Dice: 0

In [43]:
clinic_bboxes = pd.read_csv("../../data/Clinic/bounding_boxes_subject1_cd1.csv")
clinic_bboxes['image_path'] = clinic_bboxes['image_path'].apply(update_image_path)

In [45]:
clinic_image = Image.open("../../data/Clinic/Images/Subject (14).png")
clinic_image = clinic_image.convert("RGB")
np.array(clinic_image).shape

(951, 1066, 3)

In [46]:
x_scale = 256 / 951
y_scale = 256 / 1066

In [47]:
clinic_image = transforms.functional.to_tensor(clinic_image)
clinic_image = transforms.functional.resize(clinic_image, size=(256, 256), interpolation=Image.BILINEAR)

In [51]:
xmin, ymin, xmax, ymax = clinic_bboxes.query("image_path == 'Subject (14).png'")[['x1', 'y1', 'x2', 'y2']].values[0]

In [72]:
xmin = 130
xmax = 175
ymin = 30
ymax = 83

In [73]:
bbox_mask = np.zeros([1, 256, 256])
bbox_mask[ymin:ymax, xmin:xmax] = 1
bbox_mask = torch.tensor(bbox_mask, dtype=torch.float32)

In [80]:
clinic_input_image = torch.cat([clinic_image, bbox_mask], dim=0)
clinic_input_image = clinic_input_image.to(device)

In [1]:
testloop(test_loader, model, loss_func)

NameError: name 'testloop' is not defined